# Lab02: Web Crawler (Continue).

- MSSV: 18120636
- Họ và tên: Trần Ngọc Tuấn

## Yêu cầu bài tập

**Cách làm bài**


Bạn sẽ làm trực tiếp trên file notebook này; trong file, từ `TODO` để cho biết những phần mà bạn cần phải làm.

Bạn có thể thảo luận ý tưởng cũng như tham khảo các tài liệu, nhưng *code và bài làm phải là của bạn*. 

Nếu vi phạm thì sẽ bị 0 điểm cho bài tập này.

**Cách nộp bài**

Trước khi nộp bài, rerun lại notebook (`Kernel` -> `Restart & Run All`).

Bạn nộp file notebook với tên `MSSV.ipynb` của bạn (vd, nếu bạn có MSSV là 1234567 thì bạn đặt tên file là `1234567.ipynb`) và nộp file này trên moodle.

**Nội dung bài tập**

Thu thập và thể hiện dữ liệu web.

## 2. Cài đặt

### 2.1. Import library

In [2]:
import requests
import re
from bs4 import BeautifulSoup
from bs4.element import Comment
import string

### 2.2. HTML Parser


Bộ phân tích cú pháp HTML (HTML Parser): nhận HTML code và trích xuất thông tin liên quan như tiêu đề của trang, đoạn văn trong trang, tiêu đề trong trang, liên kết, văn bản in đậm, v.v.

In [3]:
from html.parser import HTMLParser

class MyHTMLParser(HTMLParser):
    def handle_starttag(self, tag, attrs):
        print("Start tag:", tag)

    def handle_endtag(self, tag):
        print("End tag :", tag)

    def handle_data(self, data):
        print("Data  :", data)

parser = MyHTMLParser()
parser.feed('<html><head><title>Test</title></head>')

Start tag: html
Start tag: head
Start tag: title
Data  : Test
End tag : title
End tag : head


#### Loại bỏ các HTML tag không cần thiết bằng cách thiết lập filter

In [4]:
def text_filter(element):
    if element.parent.name in ['style', 'title', 'script', 'head', '[document]', 'class', 'a', 'li']:
        return False
    elif isinstance(element, Comment):
        '''Opinion mining?'''
        return False
    elif re.match(r"[\s\r\n]+",str(element)): 
        '''space, return, endline'''
        return False
    return True

### 2.3 Thu thập nội dung trang Web

#### Trong bài tập này ta thể hiện tài liệu (hay nội dung trang web) với cấu trúc đơn giản như sau: 
- Gọi $D$ là một tập tài liệu chứa *n* tài liệu: $D=\left\{d_1,d_2,...,d_n\right\}$.
- Ta thể hiện tài liệu bằng một dictionary `data={}` với `data[word]=[[url_idx_1,url_idx_2,...],frequency]` với `url_index`$\in{\left[{1,n}\right]}$

VD: `data['at']=[[1,2], 5]`
Từ `at` xuất hiện trong đường dẫn có index `1` và `2` tổng số lần xuất hiện là 5.

#### Bước 1: liệt kê các từ xuất hiện trong trang web:

In [5]:
def wordList(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.content, "html.parser")
    text = soup.findAll(text=True)
    filtered_text = list(filter(text_filter, text)) # list của các chuỗi
    word_list = []

    # TODO:
    # Với mỗi chuỗi trong filtered_text:
    #   Thay thế các dấu câu thành khoảng trắng (gợi ý: danh sách các dấu câu: string.punctuation; thay thế: .replace(...))
    #   Tách chuỗi bởi khoảng trắng (.split(...))
    #   Thêm các từ vừa được tách ra vào word_list
    punc = string.punctuation 
    for i in filtered_text:
      for ele in i:
        if ele in punc:
          i=i.replace(ele," ")
      word_list.append(i.split())
    return word_list
  
# Test
print(wordList('https://en.wikipedia.org/wiki/Web_mining'))

[['Web', 'mining'], ['From', 'Wikipedia', 'the', 'free', 'encyclopedia'], ['This', 'article', 'includes', 'a'], ['related', 'reading', 'or'], [], ['but', 'its', 'sources', 'remain', 'unclear', 'because', 'it', 'lacks'], [], [], ['October', '2020'], [], [], ['This', 'article', 'may'], ['require'], ['The', 'specific', 'problem', 'is'], ['The', 'article', 'needs', 'sufficient', 'inline', 'references', 'and', 'a', 'complete', 're', 'write', 'has', 'been', 'proposed'], [], ['October', '2020'], [], [], ['Web', 'mining'], ['It', 'uses', 'automated', 'methods', 'to', 'extract', 'both', 'structured', 'and', 'unstructured', 'data', 'from', 'web', 'pages', 'server', 'logs', 'and', 'link', 'structures', 'There', 'are', 'three', 'main', 'sub', 'categories', 'of', 'web', 'mining'], ['Contents'], ['1'], ['Web', 'mining', 'types'], ['2'], ['Web', 'usage', 'mining'], ['2', '1'], ['Pros'], ['2', '2'], ['Cons'], ['3'], ['Web', 'structure', 'mining'], ['4'], ['Web', 'content', 'mining'], ['4', '1'], ['Web

#### Bước 2: Tính tần suất xuất hiện của từ trong 1 trang web, lưu trữ dữ liệu vào data:

In [7]:
def read_url(url, url_idx, data):
    word_list = wordList(url)
    # TODO
    # Với mỗi từ w trong word_list:
    #   Nếu w chưa có trong data thì khởi tạo data[w] = [[url_idx], 1]
    #   Ngược lại thì thêm url_idx vào data[w][0] (nếu chưa có) và tăng data[w][1] lên 1 đơn vị
    for i in word_list:
        for j in i:
            if j not in data:
                data[j] = [[url_idx], 1]
            else:
                if url_idx not in data[j][0]:
                    data[j][0].append(url_idx)
                data[j][1] += 1

# Test
test_data = {}
read_url('https://en.wikipedia.org/wiki/Web_mining', 1, test_data)
test_data

{'04': [[1], 1],
 '1': [[1], 7],
 '107': [[1], 1],
 '1431': [[1], 1],
 '15': [[1], 1],
 '1591404149': [[1], 1],
 '2': [[1], 6],
 '20': [[1], 1],
 '2000': [[1], 4],
 '2004': [[1], 1],
 '2005': [[1], 3],
 '2007': [[1], 1],
 '2013': [[1], 1],
 '2015': [[1], 1],
 '2019': [[1], 3],
 '2020': [[1], 3],
 '3': [[1], 1],
 '3540679813': [[1], 1],
 '4': [[1], 4],
 '5': [[1], 1],
 '55–60': [[1], 1],
 '5th': [[1], 1],
 '6': [[1], 5],
 '7281': [[1], 1],
 '978': [[1], 3],
 '9783540300175': [[1], 1],
 'A': [[1], 2],
 'About': [[1], 1],
 'An': [[1], 1],
 'Annamoradnejad': [[1], 2],
 'Anne': [[1], 1],
 'Another': [[1], 1],
 'Anthony': [[1], 1],
 'Application': [[1], 3],
 'Applications': [[1], 1],
 'April': [[1], 1],
 'Article': [[1], 1],
 'As': [[1], 1],
 'Association': [[1], 1],
 'Bauknecht': [[1], 1],
 'Berlin': [[1], 2],
 'Bibliographic': [[1], 2],
 'Blockeel': [[1], 1],
 'Books': [[1], 2],
 'By': [[1], 1],
 'Castro': [[1], 1],
 'Chinese': [[1], 2],
 'Chung': [[1], 1],
 'Cite': [[1], 2],
 'Commerce': 

#### Bước 3: Chạy chương trình lưu kết quả vào file output.txt

In [8]:
data = {}
read_url('https://en.wikipedia.org/wiki/Web_mining', 1, data)
read_url('https://en.wikipedia.org/wiki/Data_mining', 2, data)

sorted_keys = sorted(data.keys())

with open("output.txt", "w", encoding="utf-8") as f:
    output_line = "Word".ljust(20) + "Frequency".ljust(15) + "URL_idx".ljust(15) + "\n"
    f.writelines(output_line)
    f.writelines('---------------------------------------------------------------------\n\n')
    for key in sorted_keys:
        output_string = str(key).ljust(20) + str(data[key][1]).ljust(15) + str(data[key][0]).ljust(15) + "\n"
        f.writelines(output_string)


In [15]:
# TODO
# Keyword tìm kiếm: python pickle
# Dùng pickle để lưu dictionary data xuống đĩa. Sau đó đọc lên và in ra 10 giá trị đầu tiên của nó.

# 1. Lưu data
import pickle
filehandler = open('output.txt', 'wb') 
pickle.dump(data, filehandler)
# 2. Đọc lên và in ra
with open('output.txt', "rb") as f:
 temp = pickle.load(f)
print(temp)

{'Web': [[1, 2], 41], 'mining': [[1, 2], 90], 'From': [[1, 2], 3], 'Wikipedia': [[1, 2], 4], 'the': [[1, 2], 120], 'free': [[1, 2], 3], 'encyclopedia': [[1, 2], 2], 'This': [[1, 2], 9], 'article': [[1, 2], 7], 'includes': [[1], 1], 'a': [[1, 2], 31], 'related': [[1, 2], 4], 'reading': [[1, 2], 3], 'or': [[1, 2], 11], 'but': [[1, 2], 9], 'its': [[1, 2], 3], 'sources': [[1, 2], 2], 'remain': [[1], 1], 'unclear': [[1], 1], 'because': [[1, 2], 2], 'it': [[1, 2], 9], 'lacks': [[1], 1], 'October': [[1, 2], 4], '2020': [[1], 3], 'may': [[1, 2], 7], 'require': [[1], 1], 'The': [[1, 2], 33], 'specific': [[1, 2], 5], 'problem': [[1, 2], 4], 'is': [[1, 2], 42], 'needs': [[1], 3], 'sufficient': [[1, 2], 2], 'inline': [[1], 1], 'references': [[1], 3], 'and': [[1, 2], 78], 'complete': [[1], 1], 're': [[1, 2], 2], 'write': [[1], 1], 'has': [[1, 2], 8], 'been': [[1, 2], 6], 'proposed': [[1], 1], 'It': [[1, 2], 4], 'uses': [[1, 2], 5], 'automated': [[1], 1], 'methods': [[1, 2], 2], 'to': [[1, 2], 64], 